In [1]:
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [2]:
from datasets import get_data
attr, test_attr, target = get_data()

In [4]:
attr.head()

,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,...,P31,P32,P33,P34,P35,P36,P37,Year,Month,Week
0,60,0,2,4,5.0,4,4,2,2,5,...,3,4,5,5,4,3,4,1999,7,28
1,4,0,1,4,5.0,4,4,1,2,5,...,0,0,0,0,0,0,0,2008,2,7
2,14,1,2,2,4.0,2,5,2,3,5,...,0,0,0,0,0,0,0,2013,3,10
3,52,1,2,6,4.5,6,6,4,4,10,...,12,10,6,18,12,12,6,2012,2,5
4,21,1,2,3,4.0,3,4,2,2,5,...,1,3,2,3,4,3,3,2009,5,19


In [5]:
test_attr.head()

,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,...,P31,P32,P33,P34,P35,P36,P37,Year,Month,Week
0,42,1,1,1,4,4,4,1,2,5,...,0,0,0,0,0,0,0,2011,1,3
1,31,1,2,3,4,4,4,2,2,5,...,0,0,0,0,0,0,0,2011,3,11
2,4,0,1,3,4,4,4,2,2,5,...,0,0,0,0,0,0,0,2013,10,44
3,30,1,2,2,4,4,4,2,3,5,...,4,0,0,0,0,0,0,2013,5,19
4,1,1,1,2,4,4,4,1,2,5,...,0,0,0,0,0,0,0,2013,7,31


In [6]:
target.head()

0    5653753
1    6923131
2    2055379
3    2675511
4    4316715
Name: revenue, dtype: float64

In [7]:
new_reg = ExtraTreesRegressor()
new_reg.fit(attr, target)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
          verbose=0, warm_start=False)

In [8]:
results = new_reg.predict(test_attr)

In [161]:
output_df = pd.DataFrame({'Prediction': results})

In [162]:
output_df.to_csv('submission.csv')